# Search trees

In previous lectures we have spoken about associative array. Let's remember they. 

Associative array is array that works with key-objects (like string). It's interface lokks like this:

1. Construct - O(n).
2. Add element ('key','value') - add to array element with index 'key' and value. There are no exact time limits for this operation, but in the most famous implementations it take O(1) or O(logN) in worst.
3. Delete element ('key') - mark the memory with 'key' element as empty. The value will be deleted.
4. Find of \[key\] - find element with 'key' index.

So, in previous lecture we seen its implementation on hash tables. Today let's speak about another one implementation - Search trees. As before Search trees is a container. Its main idea provide guaranteed fast search through data, about O(logN). At the same time there will be good not to harm the speed of adding and removing elements. Ok, for this tasks we have hash table with asymptotics about O(1) for each operation. But there are several things in which Search tree `outperfom` Hash table:

1. Balanced (speak later about it) Search tree guaranteed the speend near O(logN) in contrast to Hash table that can work in worst situation near O(N).
2.  Search tree doesn't need a hash function and can work with original keys but the important thing is you can compare two keys (mean you can answer to question like "is a<b?").'
3. Memory - while hash table can take about 2N memory Search tree almost guaranteed take N cells of memory for data. 
4. (most important) Search tree can provide some special functions that can work with range of elements with O(logN) asymptotics. For example calculate summ for elements in range \[a,b\].

I hope that now you belive me that search trees no less important than hash tables). Let's look more closer to them.




## Binary search tree

Most simple type of such tree is simple binary search tree.

![binaryTree](images/binaryTree.png)

How you can see above it is built with some simple rules:

1. Each subtree for each node is search tree
2. For each node left child contain key less than parent node. Right child contain key bigger than parent.

That is all. In such tree to find element with some key you need about O(h) steps where h - is the hight of the tree (deep of the deepest element). To find it you just need walk from the root and at every step choose left or right element - if you key is less or equal than key into the node, than go to the left. Otherwise go to the right.



The main problem is build such a tree where the height would be minimal (logN). Ok, let's look how to build it in simplest way:

1. Add element with key K - to do this you need to search it firstly. If you find such element, just modified it. If not, at the end of search process you meet empty node (child of some element). When you meet it just create new node with element and change the link in parent to this new element.
![binaryTreeAdd](images/binaryTreeAdd.png)
2. Delete element with key K - Again trying to find the required element. If we haven't found it so we can say that it doesn't exists. Otherwise there are three cases to deal with:

        - If the element have not any child elements that we just delete it from memory and mark node as empty
        - If the element have one child than we delete element from the memory and link it parent with it child directly.
        - If the element have two childs than we need make one step to the left and then go to the right until we meet an element without right child. 
                Than we take this element, copy it data to the our node that we want to delete and delete the node at the bottom of the tree where from we copy data.

Ok, now we can construct such a tree with asympotics about O(N*k) where k - depth of the tree at the end. But still there is a worst case when we take at every iteration the element that bigger of smaller than others in the tree. Let's take a look.

In [24]:
# Native implementation with the nodes.
class Node:# Node of the tree
    left = None #Left child
    right = None #Right child
    data = None #Data that node keep inside
    key = None #Key for the data
    depth = 0

    def search (self, key):
        if (self.key == None):
            return None
        if (self.key == key):
            return self
        if (self.key > key):
            if (self.left == None): return None
            return self.left.search(key)
        else:
            if (self.right == None): return None
            return self.right.search(key)
            pass

    def add (self, key, data):
        if (self.key == None):
            self.data = data
            self.key = key
            depth = 1
            return 
        if (self.key == key):
            self.data = data
            return
        if (self.key > key):
            if (self.left == None): self.left = Node()
            self.left.add (key,data)
            self.depth = self.left.depth + 1
            return
        else:
            if (self.right == None): self.right = Node()
            self.right.add (key,data)
            self.depth = self.right.depth + 1 
            return

    def print (self):
        result = ""
        stack = [(self,None)]
        result+= str(stack) + "\n"
        currentDepth = self.depth
        while (stack):
            current = stack[0]
            #print (currentDepth)
            if (current[0].depth != currentDepth):
                
                currentDepth = current[0].depth
                result+= str(stack) + "\n"
                pass

            stack = stack[1:]
            if (current[0].left != None): stack.append((current[0].left, current[0]))
            if (current[0].right != None): stack.append((current[0].right, current[0]))
        return result

    def __repr__ (self):
        return ("(" + str (self.key) + ", " + str (self.data) + ")")


In [25]:
tree = Node()
tree.add (1,'a')
tree.add (2,'b')
tree.add (3,'c')
tree.add (4,'d')
tree.add (5,'e')
print (tree.print()) # first element is element, second is link to parent
print (tree.search (4))

[((1, a), None)]
[((2, b), (1, a))]
[((3, c), (2, b))]
[((4, d), (3, c))]
[((5, e), (4, d))]

(4, d)


Or in more simple variant - 1->2->3->4. So, our depth is equal to N because at every step we add element bigger than all previous. That is bad and we need to solve this problem.

## AVL tree

So, the first thing that comes to mind is change tree in some way after adding element. The main tasks is keep the tree height near minimum (logN). So, let's just memorise the height of the subtrees for each node. If the left/right subtree of some node deeper than right/left subtree more than 1 element (difference in 1 element can be if N is even number, for example tree with two elements 1->2) we need to balance it so that the heights are leveled.

Ok, due to the fact that we add only one element (after the add command) and taking on faith that before that the tree was balanced we have only one brach broken. Let's look at it more closer and assume that all subtrees (of node) are themselves balanced. Without loss of generality take a look into the left subtree. (for the right subtree all will be the same). Ok, the left tree became deeper than right by 2 levels (not more than 2 because we add only 1 element and it can't deepen than by 1 level). There can be three situation:

1. If the left subtree of left child is deeper (by 1) than right subtree - in this situation we can exchange places of parent and left child. Right subtree of left child become a new left subtree of parent (after it become right child). In that case the depth of L (h+1, where h depth of R) become equal to depth of C + 1 = h + 1

![avl_leftLittle](images/AVL_LL.png)

2. If the left subtree of left child depth equal to the right subtree - (actualy impossible) L = C = h + 1, this case is the same to previous one because when we move C to the right (it become left subtree for 'a') it's depth with element a become bigger than depth of L by 1 (C + 1 = h+1 + 1 = h + 2, L = h + 1)

3. If the right subtree of left child deeper (by 1) than left subtree - This case is the most difficult because if we just move 'b' to possition 'a' nothing will change and the right subtree for 'b' element become depeer than L by 2 (because C = h + 1; C + a = h+1 + 1 = h + 2 and L = h). What to do? Let's made subtree L depeer than C by the same 'rotation' as in 1 and 2 cases (but in other direction). For our example we create element 'c' that was right child of 'b' and move it up. Now 'c' become the left child of 'a' and 'b' left child of 'c'. And here we got first case where we already know how to work.

![avl_leftBig](images/AVL_BL.png)

Ok, for the mirror situations for the right subtree (when it bigger than left one) all the same. But here we use some assume that subtrees alredy balanced, but how we can achieve this? Simple, we can just use the recursion from the new element, climb up and balance the tree up to the root (from one element to it's parent) [here] (https://en.wikipedia.org/wiki/File:AVL_Tree_Example.gif) the link to good animation of this process.

Now the tree is always balanced and work with asymptotics about O(logN) for search, insert (add) and delete. However, what's interesting, using such trees you can quickly merge and splits arrays. But before we speak about it let's look to another one implementation of the search tree.

## Cartesian tree

The main idea of such tree that we can keep two keys for every node into the tree. First one is original from input data. Second one we create randomly (with uniform distribution). And with two keys we build a search tree for the first one and binary heap for the second one. Mean that for every node the first key at the left child less that in parent and right child has the key that bigger. And at the same time parent node has the second key that bigger than childs keys. It really helps to build a roughly balanced tree.

![decard](images/decard.png)

But what about the second key? If we choose it randomly why we can't just take the value as second key? In fact, we can and this is how it works. So, actualy there are only key and value. For key we build search tree to find element. For valuse we build binary heap to keep the tree balanced and have oportunity to take maximum element. On other hand this tree can transform into list in worse cases. 

![cartesian](images/cartesianTree.png)

Now let's speak about how to build such tree. Because we have at the same time a heap and search tree we can't just add element at the end of tree and rebalance/rebuild it. Actualy we need insert element at the middle of tree. And for this operation we divide our tree into two parts - part with the keys less than K and with the keys bigger than K. Than we merge part with the keys less than K with new element and result tree merge with 'right' tree with the keys bigger than K.

Ok, now about split and merge:

### Split

The main task of the split is divide array (tree) into two parts by the key 'K'. So, as in previous algorithms we can try do it recursively. We will start from the root and go down cutting the tree layer by layer. Ok, let's call the tree with the keys less or equal to  K 'left', and bigger than K 'right'. Suppose that everything above is already divided somehow. And fucntion that call separator-fucntion wait two diffrent trees, left and right. Let's create them.

Actualy we can meet three situation. First and the most simple, if we meet empty node. Then just return empty results.

Second case is when key in node less than K. In this situation we know that all elements keys in left subtree is less than key in current node. This mean that the whole left subtree should be parts of left result tree. The parts of right subtree can be less than K too (because the fact that elements in right tree bigger than our element does not mean that they are greater than the key K). That mean that in left tree we should send left subtree with current root and parts of right subtree that less than key K. But how we can find a parts of right subtree than less than K? Right - just call our separator-function for right subtree. The left result of such subtree will be the new subtree for current node. And at this moment we finish the left tree for answer. The right result from function we can just return as right answer because it is already the tree that have all keys greater than K.

As you probably guessed the third case (when current node key is grater than K) is symmetrical to the second, but we exchange left tree operations with the right one because current node K bigger then K we take a whole right tree plus some elements from the left tree.

![cartesianSplit](images/cartesianSplit.jpg)

In [ ]:
def split(tree, k):
    if tree == None:
        return (None, None)

    if (tree.key <= k):
        leftTree, rightTree = split (tree.right, k)
        tree.right = leftTree
        return (tree, rightTree)
        
    if (tree.key > k):
        leftTree, rightTree = split (tree.left, k)
        tree.left = rightTree
        return (leftTree, tree)

### Merge

Ok, now when we can split, let's look at how to merge to array. With this to function we can add,remove elements inside cartesian Tree or... merge array). However, there is a limitation for the function `merge` - it can merge only the trees one of which has all keys less than the other. Let's take two trees "left" and "right". Left tree have all keys less than right one. If it not the truth we shuld build new tree manualy by adding each element from one tree to another.

![cartesianMerge](images/cartesianMerge.jpg)

Simillar to split function we will make recursive function `merge`. First of all we take the root of the tree where the value is bigger (because, how you can remember for values there are used heap contruction). This new node will be the root of our new tree. But what about subtrees of this node? There two symmetrical cases - when we take the root from left tree or from right. Let's take a root from the left tree. In this case key of our new node is less than all keys from right tree, bigger than all keys from left subtree of left tree and less than all keys in right subtree of left tree. So we need take left subtree of left tree as new lefy subtree, and in right merge right subtree and right tree to take a merged tree with the keys grater than current. In picture - we take T1 as root T. Left subtree of T1 less than T by key so it will be the left subtree of T. Right subtree of T1 and T2 has keys grater than key T. This mean we need merge them to take a new tree. So let's just call merge on (T1.right, T2) and save result tree as T.right. That all)

In [ ]:
def merge (tree1, tree2):
    if tree1 == None: return tree2
    if tree2 == None: return tree1
    
    resultTree = Node()
    if (tree1.data >= tree2.data):
        resultTree.right = merge (tree1.right, tree2)
        resultTree.left = tree1.left
        resultTree.data = tree1.data
        resultTree.key = tree1.key

    if (tree2.data > tree1.data):
        resultTree.left = merge (tree1, tree2.left)
        resultTree.right = tree2.right
        resultTree.data = tree2.data
        resultTree.key = tree2.key
    
    return resultTree

### Interface

Now lets sum up all about Cartesian tree. Inside it contain pairs (key,value). Keys work as keys for Searh tree. Values work as keys for binary heap.

1. Split (K) - split the tree by key K into two subtree, where first one has all keys less than K and second one has all keys grater than K. O(h), h - height
2. Merge (Tree1, Tree2) - merge two tree into new one. All keys in tree 1 are less than into tree 2. O(h1+h2)
3. Search (key) - the same as for search tree. O(h)
3. Add (Key, Value) - split the tree by the Key, merge left tree with new element than merge right tree with new left. O(h)
4. Remove (Key) - search the element by the key, merge it left and right subtree and then pin the new tree instead removed element. Delete element data from memeory O(h)
5. Take biggest\lowest element from tree - O(1)

Ok, and now one interested moment - if we use array indexes as a keys for Cartesian tree we can work with usual aray that have such interface and can be merged with any other array by O(h)~O(logN). For comparison merge of two classic array take about O(N+M) operations. And in next lesson you can see some functions on the tree. 